# RCE Regress Power to Trial Outcome

Brief 1-2 sentence description of notebook.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

In [ ]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from scipy import stats
import itertools

## Inputs & Data

Explanation of each input and where it comes from.

In [ ]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

In [ ]:
MERGED_LFP_AND_SLEAP = pd.read_pickle("./proc/rce_lfp_all_window_power.pkl")

In [ ]:
OUTCOME_TO_COLOR = {'lose': "red",
 'omission': "orange",
 'rewarded': "green",
 'win': "blue"}

In [ ]:
TRIAL_OR_BASELINE_TO_STYLE = {'baseline': "--", "trial": "-"}

In [ ]:
# Change this to filter for different velocities
# 0: 0 to 2.5cm/s
# 1: 2.5 to 5cm/s
# 2: 5 to 10cm/s
# 3: 10+cm/s
FILTERING_VELOCITY = [0, 1]

In [ ]:
BAND_TO_FREQUENCY = {"theta": (4,13), "beta": (12, 30), "gamma": (30, 90)}

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [ ]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [ ]:
MERGED_LFP_AND_SLEAP.head()

In [ ]:
for col in MERGED_LFP_AND_SLEAP.columns:
    print(col)

## Getting the average power for each frequency band from power with all windows

In [ ]:
all_windows_columns = [col for col in MERGED_LFP_AND_SLEAP.columns if "all_window_averaged_power" in col]


In [ ]:
for col in all_windows_columns:
    region_and_baseline_or_trial = "_".join(col.split("_")[:2])
    print(region_and_baseline_or_trial)
    for band, (freq_start, freq_end) in BAND_TO_FREQUENCY.items():
        band_col = "{}_{}_all_not_parsed_power".format(region_and_baseline_or_trial, band)
        MERGED_LFP_AND_SLEAP[band_col] = MERGED_LFP_AND_SLEAP[col].apply(lambda x: x[freq_start:freq_end])
        MERGED_LFP_AND_SLEAP["{}_{}_averaged_not_parsed_power".format(region_and_baseline_or_trial, band)] = MERGED_LFP_AND_SLEAP[band_col].apply(lambda x: x.mean())

In [ ]:
MERGED_LFP_AND_SLEAP.head()

## Filtering power windows based on velocity

- Getting a mask for each segment based on velocity

In [ ]:
MERGED_LFP_AND_SLEAP["binned_trial_velocity_mask"] = MERGED_LFP_AND_SLEAP["binned_trial_thorax_velocity"].apply(lambda x: [index for index, num in enumerate(x) if num in FILTERING_VELOCITY])

MERGED_LFP_AND_SLEAP["binned_baseline_velocity_mask"] = MERGED_LFP_AND_SLEAP["binned_baseline_thorax_velocity"].apply(lambda x: [index for index, num in enumerate(x) if num in FILTERING_VELOCITY])

- Filtering out the power windows based on windows that have the filtered velocity

In [ ]:
baseline_windowed_power_columns = [col for col in MERGED_LFP_AND_SLEAP.columns if "baseline_all_window_power" in col]
trial_windowed_power_columns = [col for col in MERGED_LFP_AND_SLEAP.columns if "trial_all_window_power" in col]

In [ ]:
baseline_windowed_power_columns

In [ ]:
for col in baseline_windowed_power_columns:
    region_and_baseline_or_trial = "_".join(col.split("_")[:2])
    print(region_and_baseline_or_trial)
    MERGED_LFP_AND_SLEAP["{}_velocity_parsed_window_averaged_power".format(region_and_baseline_or_trial)] = MERGED_LFP_AND_SLEAP.apply(lambda x: np.nanmean(np.array([x[col][i] for i in x["binned_baseline_velocity_mask"]]), axis=0), axis=1)

for col in trial_windowed_power_columns:
    region_and_baseline_or_trial = "_".join(col.split("_")[:2])
    print(region_and_baseline_or_trial)
    MERGED_LFP_AND_SLEAP["{}_velocity_parsed_window_averaged_power".format(region_and_baseline_or_trial)] = MERGED_LFP_AND_SLEAP.apply(lambda x: np.nanmean(np.array([x[col][i] for i in x["binned_trial_velocity_mask"]]), axis=0), axis=1)

In [ ]:
velocity_parsed_columns = [col for col in MERGED_LFP_AND_SLEAP.columns if "velocity_parsed_window_averaged_power" in col]

In [ ]:
for col in velocity_parsed_columns:
    region_and_baseline_or_trial = "_".join(col.split("_")[:2])
    print(region_and_baseline_or_trial)
    for band, (freq_start, freq_end) in BAND_TO_FREQUENCY.items():
        band_col = "{}_{}_all_velocity_parsed_power".format(region_and_baseline_or_trial, band)
        MERGED_LFP_AND_SLEAP[band_col] = MERGED_LFP_AND_SLEAP[col].apply(lambda x: x[freq_start:freq_end])
        MERGED_LFP_AND_SLEAP["{}_{}_averaged_velocity_parsed_power".format(region_and_baseline_or_trial, band)] = MERGED_LFP_AND_SLEAP[band_col].apply(lambda x: x.mean())

In [ ]:
MERGED_LFP_AND_SLEAP.head()

# Regressing with power of all windows

In [ ]:
baseline_averaged_all_power_columns = [col for col in MERGED_LFP_AND_SLEAP.columns if "averaged_not_parsed_power" in col and "baseline" in col]
trial_averaged_all_power_columns = [col for col in MERGED_LFP_AND_SLEAP.columns if "averaged_not_parsed_power" in col and "trial" in col]

In [ ]:
MERGED_LFP_AND_SLEAP[["trial_outcome", "current_subject"] + trial_averaged_all_power_columns].head()

# Regressing with power of all windows

In [ ]:
baseline_velocity_all_power_columns = [col for col in MERGED_LFP_AND_SLEAP.columns if "averaged_velocity_parsed_power" in col and "baseline" in col]
trial_velocity_all_power_columns = [col for col in MERGED_LFP_AND_SLEAP.columns if "averaged_velocity_parsed_power" in col and "trial" in col]

In [ ]:
MERGED_LFP_AND_SLEAP[["trial_outcome", "current_subject"] + trial_velocity_all_power_columns].head()